# Intro to geomeppy_tutorial
----
#### Addition to Eppy, GeomEppy is a scripting language to Energyplus, primarily focuses on IDF geometry (zones, surfaces, constructions, etc.)
#### This tutorial is intended to help you take the first step in using GeomEppy. The demo file is a quick introduction of using GeomEppy to build geometries for idf files:
* access an idf file with geomeppy
* build a geometry for the idf file
* visualize the built geometry
* modified the built geomety: scaling, rotating, etc. 
* run energy plus on geomeppy

# Install geomeppy:
--------
In your **conda** environment, run:

In [ ]:
pip install geomeppy

Next install **energy plus** if you have not done so. (This tutorial will be running with EnergyPlus 9.1.0)
Get the idd filepath from the EnergyPlus installation folder. The file for this tutorial is on Mac, so the idd filepath would be: '/Applications/EnergyPlus-9-1-0/Energy+.idd'. 
On windows the path would usually be: 'C:/EnergyPlusV9-1-0', and on Linux '/usr/local/EnergyPlus-9-1-0'.

Next, we are going to import idf file to **geomeppy**. 
Get the **idd file** path mentioned above ready'/Applications/EnergyPlus-9-1-0/Energy+.idd'.
Pick an **idf file** that you wish you access. This tutorial uses the 'Minimal.idf' file from the energy plus example folder: '/Applications/EnergyPlus-9-1-0/ExampleFiles/Minimal.idf'. 
With these ready, in the python 3 environment, type: 

In [ ]:
from geomeppy import IDF
IDF.setiddname('/Applications/EnergyPlus-9-1-0/Energy+.idd')
idf = IDF('/Applications/EnergyPlus-9-1-0/ExampleFiles/Minimal.idf')

Then we can add a a weather file to our idf object. This tutorial uses 'USA_CO_Golden-NREL.724666_TMY3.epw'

In [ ]:
idf.epw = 'USA_CO_Golden-NREL.724666_TMY3.epw'

Now we could start adding a simple block to the idf object by giving it a name, specifying its four base coordinates, and its height. 

In [ ]:
idf.add_block(
    name='Test Block1',
    coordinates=[(10,0),(10,5),(0,5),(0,0)],
    height=5)

The block could be added with more details, such as number of floor and undergournd stories.

In [ ]:
idf.add_block(
    name='Test Block2',
    coordinates=[(10,5),(10,10),(0,10),(0,5)],
    height=20,
    num_stories=5,
    below_ground_stories=1,
    below_ground_storey_height=5,)

We could also add windows to the block by giving it window to wall ratio(wwr)

In [ ]:
idf.set_wwr(wwr=0.25)

The blocks could be rotated:

In [ ]:
idf.rotate(90)

And scaled on selected axis, here we double the dimension of the blocks on 'zy' axis:

In [ ]:
idf.scale(2,axes='zy')

We could use the **view_model** function to view the built geometry

In [ ]:
idf.view_model()

Intersect all surfaces:

In [ ]:
idf.intersect_match()

Next we can set default constructions for the idf surfaces.

In [ ]:
idf.set_default_constructions()

We can view the idf file and check the modification you made to the idf file. 

In [ ]:
idf.printidf()

Finally, we can run the idf file on geomeppy

In [ ]:
idf.run()

The idf file could be saved or saved as:

In [ ]:
idf.save('Filepath to save the file')

In [ ]:
idf.saveas('Filepath to set the new idf file name attribute to and save the file as')

# more geomeppy resources
##### The source documentation of geomeppy is here: https://geomeppy.readthedocs.io/en/latest/